Analyzing Crime Statistics for the Summer of 2014 in Seattle
============================================================

This is an exploration of (a reduced set of) public crime data in the city of Seattle during the Summer of 2014. The goal is to identify and visualize interesting patterns in this data.

The Data
--------

The data comes in the form of a CSV file, so let's load it up in [Pandas](http://pandas.pydata.org/pandas-docs/stable/) and see what we've got.

In [2]:
import pandas as pd

In [3]:
seattle_data = pd.read_csv('data/seattle_incidents_summer_2014.csv')

/Users/hank/dev/.virtualenvs/datasci_assignment_six/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Well, the first I noticed about it is that type inference is failing on column 9. Let's see what it's supposed to be.

In [6]:
seattle_data.columns

Index(['RMS CDW ID', 'General Offense Number', 'Offense Code',
       'Offense Code Extension', 'Offense Type', 'Summary Offense Code',
       'Summarized Offense Description', 'Date Reported',
       'Occurred Date or Date Range Start', 'Occurred Date Range End',
       'Hundred Block Location', 'District/Sector', 'Zone/Beat',
       'Census Tract 2000', 'Longitude', 'Latitude', 'Location', 'Month',
       'Year'],
      dtype='object')

Apparently, that's `Occurred Date Range End`. Let's take a peak and see what's in that column.

In [8]:
seattle_data['Occurred Date Range End']

0        06/28/2015 10:31:00 AM
1        06/24/2015 11:09:00 AM
2                           NaN
3                           NaN
4        11/01/2014 12:00:00 PM
5        07/10/2014 02:45:00 PM
6        06/01/2015 06:15:00 PM
7        03/15/2015 05:30:00 PM
8                           NaN
9                           NaN
10       03/27/2015 12:00:00 PM
11                          NaN
12       04/22/2015 12:00:00 AM
13       04/22/2015 12:00:00 AM
14       04/19/2015 10:00:00 AM
15                          NaN
16       08/03/2014 04:30:00 PM
17       04/08/2015 10:30:00 PM
18       04/06/2015 12:00:00 PM
19       04/06/2015 12:30:00 PM
20       03/24/2015 09:00:00 AM
21       03/14/2015 12:00:00 AM
22       03/18/2015 08:20:00 AM
23       09/30/2014 11:59:00 PM
24       06/21/2014 03:00:00 PM
25       03/08/2015 09:42:00 PM
26       08/25/2014 07:00:00 AM
27                          NaN
28       06/20/2014 08:00:00 AM
29       02/23/2015 03:00:00 PM
                  ...          
32749   

OK, it seems like the problem is some `NaN` entries. Luckily, Pandas let's us specify different strings that mean N/A, so we'll read the file again and tell it that `NaN` is one such value.

While we're at it, let's tell Pandas to parse all these different timestamps. We'll leave it to Pandas to figure out the format, if possible.

In [10]:
seattle_data = pd.read_csv(
    'data/seattle_incidents_summer_2014.csv',  # The data file.
    na_values=['NaN'],  # 'NaN' means it's a missing value.
    parse_dates=[7,8,9],  # Columns 7-9 contain timestamps; try to parse them as such.
    infer_datetime_format=True  # I don't want to comb over all that data to figure out a format; let Pandas try.
)

No warnings or errors this time, so I suspect our changes worked. Let's go ahead and peek at the data to see how it looks.

In [13]:
seattle_data

,RMS CDW ID,General Offense Number,Offense Code,Offense Code Extension,Offense Type,Summary Offense Code,Summarized Offense Description,Date Reported,Occurred Date or Date Range Start,Occurred Date Range End,Hundred Block Location,District/Sector,Zone/Beat,Census Tract 2000,Longitude,Latitude,Location,Month,Year
0,483839,2015218538,2202,0,BURGLARY-FORCE-RES,2200,BURGLARY,2015-06-28 10:31:00,2014-06-28 10:31:00,2015-06-28 10:31:00,6XX BLOCK OF NW 74 ST,J,J2,2900.3013,-122.364672,47.682524,"(47.68252427, -122.364671996)",6,2014
1,481252,2015213067,2610,0,FRAUD-IDENTITY THEFT,2600,FRAUD,2015-06-24 11:09:00,2014-06-01 00:00:00,2015-06-24 11:09:00,23XX BLOCK OF 43 AV E,C,C2,6300.1004,-122.277080,47.639901,"(47.639900761, -122.277080248)",6,2014
2,481375,2015210301,2316,0,THEFT-MAIL,2300,MAIL THEFT,2015-06-22 09:22:00,2014-08-31 09:00:00,NaT,81XX BLOCK OF 11 AV SW,F,F3,11300.5013,-122.349312,47.529232,"(47.529232299, -122.349312181)",8,2014
3,481690,2015209327,2599,0,COUNTERFEIT,2500,COUNTERFEIT,2015-06-21 15:52:00,2014-06-20 13:38:00,NaT,6XX BLOCK OF PINE ST,M,M2,8200.1002,-122.334818,47.612368,"(47.612368448, -122.334817763)",6,2014
4,478198,2015207880,2399,3,THEFT-OTH,2300,OTHER PROPERTY,2015-06-20 11:59:00,2014-06-01 11:59:00,2014-11-01 12:00:00,77XX BLOCK OF SUNNYSIDE AV N,J,J3,2700.2015,-122.329379,47.685960,"(47.685959879, -122.329378505)",6,2014
5,480485,2015904103,2308,0,THEFT-BUILDING,2300,OTHER PROPERTY,2015-06-19 14:55:00,2014-06-19 14:45:00,2014-07-10 14:45:00,35XX BLOCK OF S FERDINAND ST,R,R3,10300.4006,-122.287478,47.557855,"(47.557854802, -122.287477902)",6,2014
6,470170,2015185464,2308,0,THEFT-BUILDING,2300,OTHER PROPERTY,2015-06-04 11:13:00,2014-06-01 00:00:00,2015-06-01 18:15:00,24XX BLOCK OF WESTVIEW DR W,Q,Q2,5900.4035,-122.370773,47.640095,"(47.640094502, -122.370772861)",6,2014
7,465137,2015174988,2605,0,FRAUD-CREDIT CARD,2600,FRAUD,2015-05-27 15:45:00,2014-08-22 11:00:00,2015-03-15 17:30:00,22XX BLOCK OF NW 59 ST,B,B1,4700.4002,-122.386164,47.671643,"(47.671642511, -122.386164453)",8,2014
8,461710,2015168191,2610,0,FRAUD-IDENTITY THEFT,2600,FRAUD,2015-05-22 10:45:00,2014-08-01 00:00:00,NaT,11XX BLOCK OF REPUBLICAN ST,D,D3,7300.1019,-122.333647,47.623119,"(47.62311893, -122.333646878)",8,2014
9,456091,2015157266,2606,1,FRAUD-CHECK,2600,FRAUD,2015-05-13 17:10:00,2014-07-20 12:00:00,NaT,31XX BLOCK OF WESTERN AV,D,D1,8001.3006,-122.356376,47.618029,"(47.618028638, -122.356375788)",7,2014


That seems reasonable, so let's move on to the next step: visualizing the data and identifying patterns.

Analysis
--------

Our dataset, when augmented with some external datasets, can answer a wide range of questions. Let's choose a few for this exercise.

### Questions to Answer

1. How do incidents vary by time of day?
    1. Which incidents are most common in the evening?
    1. During what periods of the day are robberies most common?

1. How do incidents vary by neighborhood?
    1. Which incidents are most common in the city center?
    1. In what areas or neighborhoods are robberies or thefts most common?

1. How do incidents vary month to month in the Summer 2014 dataset?

1. Which incident types tend to correlate with each other on a day-by-day basis?

1. Do certain crimes correlate with environmental factors such as temperature? NOTE: This question cannot be answered without incorporating additional data. I have reached out to a potential source of weather data, but have not yet heard back from them.

### Question 1: How does the overall incident rate vary by time of day?

To answer this question, we'll star by plotting the total number of incidents by date over the full dataset.

In [18]:
# Tell Jupyter we want to render these plots inline in the notebook.
%matplotlib inline
# Now import seaborn to make things prettier.
import seaborn

Uh-oh. Looking at our `DataFrame`, we can see that the 'Occurred Date or Date Range Start' is actually a timestamp. For our purposes, we're going to want to roll up by date, so let's add a new column, `date_of_incident`, and then use [`groupby`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) on it.

In [87]:
import datetime

def get_date(ts):
    return ts.date()

with_date = seattle_data.assign(
    date_of_incident = seattle_data['Occurred Date or Date Range Start'].apply(get_date),
    identity = 1
)
date_and_offense_counts = with_date[["date_of_incident", "Offense Type", "identity"]].groupby(["date_of_incident", "Offense Type"])
date_and_offense_counts.groups

{(Timestamp('2014-06-28 00:00:00'), 'COUNTERFEIT'): [22924, 22925],
 (Timestamp('2014-08-15 00:00:00'), 'WARRARR-FELONY'): [6378,
  6379,
  6453,
  6454],
 (Timestamp('2014-07-13 00:00:00'), 'DISTURBANCE-OTH'): [17977,
  17979,
  18239,
  18242,
  18342,
  18343,
  18442,
  18443,
  18488,
  18489],
 (Timestamp('2014-06-25 00:00:00'), 'ASSLT-AGG-BODYFORCE'): [24324, 24325],
 (Timestamp('2014-06-15 00:00:00'), 'ROBBERY-STREET-WEAPON'): [28053, 28056],
 (Timestamp('2014-07-17 00:00:00'), 'THEFT-CARPROWL'): [11524,
  11525,
  13272,
  13802,
  13804,
  16225,
  16227,
  16287,
  16288,
  16301,
  16302,
  16348,
  16349,
  16405,
  16406,
  16419,
  16420,
  16437,
  16438,
  16441,
  16442,
  16449,
  16450,
  16452,
  16453,
  16457,
  16458,
  16470,
  16472,
  16503,
  16504,
  16511,
  16512,
  16523,
  16548,
  16550,
  16575,
  16576,
  16580,
  16581,
  16582,
  16602,
  16617,
  16618,
  16625,
  16626,
  16664,
  16665,
  16676,
  16677,
  16684,
  16685,
  16803,
  16804,
  168